# Библиотеки

In [1]:
import neptune
import pandas as pd

from cv import get_indices
from load_data import load
from model_selection import training
import random
from NN import simple_torchpl
from pl_framework import nn_training

In [2]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [3]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="6eddcb21-b0fa-4073-a5c6-f26572800a8b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [4]:
data_path = 'data/'
random_state = 54321
features = ['A_rate', 'A_CH4', 'A_C2H6', 'A_C3H8', 'A_iC4H10', 'A_nC4H10',
            'A_iC5H12', 'A_nC5H12', 'A_C6H14', 'B_rate']

# Загрузка данных

In [5]:
train_features, train_targets, _ = load(data_path)

In [6]:
train_features = train_features[(train_features['timestamp'] > pd.to_datetime('2020-02-15 00:00:00'))].reset_index(drop = True)
train_targets = train_targets[(train_targets['timestamp'] > pd.to_datetime('2020-02-15 00:00:00'))].reset_index(drop = True)

# Валидация

In [7]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
#                                 (pd.to_datetime('2020-03-31 00:00:00'), pd.to_datetime('2020-04-15 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [8]:
first_train = cv[0][0]
t = 0
for fold in cv:
    fold[0] = first_train
    t = t+1

In [9]:
train_targets = train_targets.drop('timestamp', axis = 'columns')

In [10]:
train_features

,timestamp,A_rate,A_CH4,A_C2H6,A_C3H8,A_iC4H10,A_nC4H10,A_iC5H12,A_nC5H12,A_C6H14,B_rate
0,2020-02-15 00:30:00,69.915537,0.114306,4.287558,39.297010,17.548963,23.866907,4.756306,4.463431,5.627749,71.291920
1,2020-02-15 01:00:00,69.896451,0.118232,4.343639,39.247930,17.442403,23.790457,4.781054,4.529383,5.708145,71.387558
2,2020-02-15 01:30:00,69.839578,0.119258,4.441232,39.256640,17.353305,23.716155,4.771902,4.543509,5.757802,71.281107
3,2020-02-15 02:00:00,70.016886,0.117634,4.513587,39.288480,17.297076,23.692166,4.772328,4.534026,5.742425,71.258247
4,2020-02-15 02:30:00,69.843409,0.116812,4.485880,39.270227,17.342867,23.749470,4.775587,4.511268,5.704208,71.320995
...,...,...,...,...,...,...,...,...,...,...,...
3642,2020-04-30 21:30:00,72.009730,0.284151,4.936343,35.758852,16.473834,24.318968,6.069360,5.527463,6.577563,74.183355
3643,2020-04-30 22:00:00,71.854180,0.282501,4.960453,35.877193,16.448487,24.074193,5.981357,5.407256,6.915378,74.213066
3644,2020-04-30 22:30:00,71.783273,0.277686,5.013878,36.271557,16.500147,24.010123,5.829363,5.160329,6.884173,74.144367
3645,2020-04-30 23:00:00,71.717743,0.271106,5.080144,36.616673,16.559533,24.008848,5.750492,5.017941,6.642076,74.320575


# Подбор модели

In [11]:
train_features.shape

(3647, 11)

In [12]:
train_targets.shape

(3647, 4)

In [ ]:
my_training = training(name = 'NN', nn_model = simple_torchpl, training_nn = nn_training,
                       description = 'Ffill. 1 train set for all folds. 4 linear layers. \
                       3 drops. Loss = MSE. No normalisation. Only first 2 fold and test set(last fold).Dropped data before 2020-02-15 00:00:00',
                       upload_source_files = ['cv.py', 
                                              'load_data.py', 
                                              'model_selection.py',
                                              'NN.py',
                                              'pl_framework.py'])
my_training.set_up_studying(random_state = random_state)

model = 'torch'

def params_func(trial, X):
    return(
        {'n_in': trial.suggest_int('n_in', 0, 60),
         'n_h_1': trial.suggest_int('n_h_1', 2, 2048),
         'n_h_2': trial.suggest_int('n_h_2', 2, 2048),
         'n_h_3': trial.suggest_int('n_h_2', 2, 2048),
         'batch_size': trial.suggest_int('batch_size', 10, 3000),
         'p_1': trial.suggest_uniform('p_1', 0, 1),
         'p_2':trial.suggest_uniform('p_2', 0, 1),
         'p_3': trial.suggest_uniform('p_3', 0, 1),
         'n_out': 4,
         'activation1': trial.suggest_categorical('activation1', ['Tanh','Hardtanh','Hardshrink', 'ELU' , 
                                                                  'SELU', 'ReLU', 'Tanhshrink', 'CELU']),
         'activation2': trial.suggest_categorical('activation2', ['Tanh','Hardtanh','Hardshrink', 'ELU' , 
                                                                  'SELU', 'ReLU', 'Tanhshrink', 'CELU']),
         'activation3': trial.suggest_categorical('activation3', ['Tanh','Hardtanh','Hardshrink', 'ELU' , 
                                                                  'SELU', 'ReLU', 'Tanhshrink', 'CELU']),
         'optimizer': trial.suggest_categorical('optimizer', ['AdamW','Adadelta','Adagrad',
                                                               'Adam','Adamax',
                                                              'ASGD','LBFGS',
                                                              'RMSprop','Rprop',
                                                              'SGD']),
         'lr': trial.suggest_loguniform('lr', 0.0001, 1.0),
         'weight_decay': trial.suggest_uniform('weight_decay', 0.001, 1.0)}
    )

n_trials = 150
my_training.train(X = train_features, 
                  y = train_targets, 
                  cv = cv, 
                  model=model, 
                  params_func = params_func, 
                  n_trials = n_trials)

https://ui.neptune.ai/iliaavilov/SIBUR/e/SIB-79


[I 2020-11-26 16:18:22,854] A new study created in memory with name: no-name-31420a7b-5eea-4a9a-9d64-3410344846fa


current params: {'n_in': 17, 'n_h_1': 652, 'n_h_2': 1052, 'n_h_3': 1052, 'batch_size': 106, 'p_1': 0.7918038353977315, 'p_2': 0.4297847966684789, 'p_3': 0.5430129619658469, 'n_out': 4, 'activation1': 'CELU', 'activation2': 'Hardshrink', 'activation3': 'SELU', 'optimizer': 'AdamW', 'lr': 0.12895914891146487, 'weight_decay': 0.48946073343376184}
Int64Index([ 17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
            ...
            709, 710, 711, 712, 713, 714, 715, 716, 717, 718],
           dtype='int64', length=702)
(3630, 180)
(3630, 4)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name         | Type       | Params
---------------------------------------------
0  | activ1       | CELU       | 0     
1  | activ2       | Hardshrink | 0     
2  | activ3       | SELU       | 0     
3  | dropout1     | Dropout    | 0     
4  | dropout2     | Dropout    | 0     
5  | dropout3     | Dropout    | 0     
6  | linear1      | Linear     | 118 K 
7  | linear2      | Linear     | 686 K 
8  | linear3      | Linear     | 1 M   
9  | linear_final | Linear     | 4 K   
10 | loss         | MSELoss    | 0     
11 | loss_sec     | MAPE       | 0     


train_loss: 1.3540476560592651
train_loss: 123129.625
train_loss: 1455883.125
train_loss: 9225.6435546875
train_loss: 38564.31640625
train_loss: 59320.01953125
train_loss: 45580.5390625
val_loss: 20042.228515625
train_loss: 33156.46875
train_loss: 29565.69140625
train_loss: 79030.6953125
train_loss: 56653.10546875
train_loss: 68918.1328125
train_loss: 54054.51953125
train_loss: 47213.83984375
val_loss: 2991.26806640625
train_loss: 47850.3671875
train_loss: 28620.158203125
train_loss: 28202.22265625
train_loss: 21757.953125
train_loss: 19477.87109375
train_loss: 19086.53515625
train_loss: 12559.3740234375
val_loss: 774.449951171875
train_loss: 8794.12109375
train_loss: 11138.2607421875
train_loss: 7459.8291015625
train_loss: 6079.96337890625
train_loss: 5840.5380859375
train_loss: 6746.02587890625
train_loss: 5110.56787109375
val_loss: 4528.3837890625
train_loss: 4507.5419921875
train_loss: 3696.811279296875
train_loss: 2406.889404296875
train_loss: 2005.66064453125
train_loss: 1742.157

...

val_loss: 0.4704418480396271
train_loss: 8.691157341003418
val_loss: 0.790660560131073
train_loss: 8.659453392028809
val_loss: 0.6850029826164246
train_loss: 8.392772674560547


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


val_loss: 0.5621100068092346



   | Name         | Type       | Params
---------------------------------------------
0  | activ1       | ELU        | 0     
1  | activ2       | SELU       | 0     
2  | activ3       | Hardshrink | 0     
3  | dropout1     | Dropout    | 0     
4  | dropout2     | Dropout    | 0     
5  | dropout3     | Dropout    | 0     
6  | linear1      | Linear     | 228 K 
7  | linear2      | Linear     | 862 K 
8  | linear3      | Linear     | 4 M   
9  | linear_final | Linear     | 8 K   
10 | loss         | MSELoss    | 0     
11 | loss_sec     | MAPE       | 0     


train_loss: 1.116101622581482
val_loss: 37993.90234375
train_loss: 44463.33984375
val_loss: 26401.986328125
train_loss: 63442.31640625
val_loss: 645733.125
train_loss: 756355.625
val_loss: 2249536.75
train_loss: 2576584.25
val_loss: 356962.03125
train_loss: 395881.0625
val_loss: 1261733.875
train_loss: 1460844.25
val_loss: 429767.65625
train_loss: 492247.40625
val_loss: 5033.1826171875
train_loss: 13689.0732421875
val_loss: 3005.101318359375
train_loss: 3472.213623046875
val_loss: 233.8521728515625
train_loss: 2258.97900390625
val_loss: 414.26611328125
train_loss: 432.7801818847656
val_loss: 220.44869995117188
train_loss: 230.32266235351562
val_loss: 41.27610778808594
train_loss: 483.258056640625
val_loss: 128.26947021484375
train_loss: 240.6203155517578
val_loss: 108.62325286865234
train_loss: 143.19033813476562
val_loss: 145.1155548095703
train_loss: 173.9439239501953
val_loss: 97.79513549804688
train_loss: 118.52783203125
val_loss: 57.64085388183594
train_loss: 69.21502685546875
val

In [ ]:
#neptune.log_metric('leaderboard_mape', 3.1250)
#neptune.project.get_experiments('SIB-68')[0].stop()
#neptune.stop()